In [1]:
#!/usr/bin/env python
# Module     : SysTrayIcon.py
# Synopsis   : Windows System tray icon.
# Programmer : Simon Brunning - simon@brunningonline.net
# Date       : 11 April 2005
# Notes      : Based on (i.e. ripped off from) Mark Hammond's
#              win32gui_taskbar.py and win32gui_menu.py demos from PyWin32
'''TODO

For now, the demo at the bottom shows how to use it...'''
         
import os
import sys
import win32api
import win32con
import win32gui_struct
try:
    import winxpgui as win32gui
except ImportError:
    import win32gui

class SysTrayIcon(object):
    '''TODO'''
    QUIT = 'QUIT'
    SPECIAL_ACTIONS = [QUIT]
    
    FIRST_ID = 1023
    
    def __init__(self,
                 icon,
                 hover_text,
                 static_options,
                 left_click_callback=None,
                 addition_menu_callback=None,
                 on_quit=None,
                 default_menu_index=None,
                 window_class_name=None,):
        
        self.icon = icon
        self.hover_text = hover_text
        self.on_quit = on_quit
        self.LCC = left_click_callback
        self.AMC = addition_menu_callback
        
        static_options = static_options + (('Quit', None, self.QUIT),)
        self._next_action_id = self.FIRST_ID
        self.static_actions = dict()
        self.static_options = self._add_ids_to_menu_options(list(static_options), self.static_actions)
        
        
        self.default_menu_index = (default_menu_index or 0)
        self.window_class_name = window_class_name or "SysTrayIconPy"
        
        message_map = {win32gui.RegisterWindowMessage("TaskbarCreated"): self.restart,
                       win32con.WM_DESTROY: self.destroy,
                       win32con.WM_COMMAND: self.command,
                       win32con.WM_USER+20 : self.notify,}
        if not SysTrayIcon.__dict__.has_key('classAtom'):
            # Register the Window class.
            window_class = win32gui.WNDCLASS()
            SysTrayIcon.hinst = window_class.hInstance = win32gui.GetModuleHandle(None)
            window_class.lpszClassName = self.window_class_name
            window_class.style = win32con.CS_VREDRAW | win32con.CS_HREDRAW;
            window_class.hCursor = win32gui.LoadCursor(0, win32con.IDC_ARROW)
            window_class.hbrBackground = win32con.COLOR_WINDOW
            window_class.lpfnWndProc = message_map # could also specify a wndproc.
            SysTrayIcon.classAtom = win32gui.RegisterClass(window_class)
        # Create the Window.
        style = win32con.WS_OVERLAPPED | win32con.WS_SYSMENU
        self.hwnd = win32gui.CreateWindow(SysTrayIcon.classAtom,
                                          self.window_class_name,
                                          style,
                                          0,
                                          0,
                                          win32con.CW_USEDEFAULT,
                                          win32con.CW_USEDEFAULT,
                                          0,
                                          0,
                                          SysTrayIcon.hinst,
                                          None)
        win32gui.UpdateWindow(self.hwnd)
        self.notify_id = None
        self.refresh_icon()

    def _add_ids_to_menu_options(self, menu_options, menu_actions_by_id):
        result = []
        for menu_option in menu_options:
            option_text, option_icon, option_action = menu_option
            if callable(option_action) or option_action in self.SPECIAL_ACTIONS:
                menu_actions_by_id[self._next_action_id] = option_action
                result.append(menu_option + (self._next_action_id,))
            elif non_string_iterable(option_action):
                result.append((option_text,
                               option_icon,
                               self._add_ids_to_menu_options(option_action, menu_actions_by_id),
                               self._next_action_id))
            else:
                print 'Unknown item', option_text, option_icon, option_action
            self._next_action_id += 1
        return result
        
    def refresh_icon(self):
        # Try and find a custom icon
        hinst = win32gui.GetModuleHandle(None)
        if os.path.isfile(self.icon):
            icon_flags = win32con.LR_LOADFROMFILE | win32con.LR_DEFAULTSIZE
            hicon = win32gui.LoadImage(hinst,
                                       self.icon,
                                       win32con.IMAGE_ICON,
                                       0,
                                       0,
                                       icon_flags)
        else:
            hicon = win32gui.LoadIcon(0, win32con.IDI_APPLICATION)

        if self.notify_id: message = win32gui.NIM_MODIFY
        else: message = win32gui.NIM_ADD
        self.notify_id = (self.hwnd,
                          0,
                          win32gui.NIF_ICON | win32gui.NIF_MESSAGE | win32gui.NIF_TIP,
                          win32con.WM_USER+20,
                          hicon,
                          self.hover_text)
        win32gui.Shell_NotifyIcon(message, self.notify_id)
        
    def refresh_icon2(self, hicon, hover_text):
        if self.notify_id: message = win32gui.NIM_MODIFY
        else: message = win32gui.NIM_ADD
        self.notify_id = (self.hwnd,
                          0,
                          win32gui.NIF_ICON | win32gui.NIF_MESSAGE | win32gui.NIF_TIP,
                          win32con.WM_USER+20,
                          hicon,
                          hover_text)
        win32gui.Shell_NotifyIcon(message, self.notify_id)

    def restart(self, hwnd, msg, wparam, lparam):
        self.refresh_icon()

    def destroy(self, hwnd, msg, wparam, lparam):
        if self.on_quit: self.on_quit(self)
        nid = (self.hwnd, 0)
        win32gui.Shell_NotifyIcon(win32gui.NIM_DELETE, nid)
        win32gui.PostQuitMessage(0) # Terminate the app.

    def notify(self, hwnd, msg, wparam, lparam):
        if lparam==win32con.WM_LBUTTONDBLCLK:
            pass
            # self.execute_menu_option(self.default_menu_index + self.FIRST_ID)
        elif lparam==win32con.WM_RBUTTONUP:
            self.show_menu()
        elif lparam==win32con.WM_LBUTTONUP:
            if callable(self.LCC):
                self.LCC()
        return True
        
    def show_menu(self):
        menu = win32gui.CreatePopupMenu()
        dynamic_actions = dict()
        dynamic_options = []
        if self.AMC:
            dynamic_options = self._add_ids_to_menu_options(list(self.AMC()), dynamic_actions)
        self.all_actions = {}
        self.all_actions.update(self.static_actions)
        self.all_actions.update(dynamic_actions)
        self.create_menu(menu, dynamic_options + self.static_options)
        #win32gui.SetMenuDefaultItem(menu, 1000, 0)
        
        pos = win32gui.GetCursorPos()
        # See http://msdn.microsoft.com/library/default.asp?url=/library/en-us/winui/menus_0hdi.asp
        win32gui.SetForegroundWindow(self.hwnd)
        win32gui.TrackPopupMenu(menu,
                                win32con.TPM_LEFTALIGN,
                                pos[0],
                                pos[1],
                                0,
                                self.hwnd,
                                None)
        win32gui.PostMessage(self.hwnd, win32con.WM_NULL, 0, 0)
    
    def create_menu(self, menu, menu_options):
        for option_text, option_icon, option_action, option_id in menu_options[::-1]:
            if option_icon:
                option_icon = self.prep_menu_icon(option_icon)
            
            if option_id in self.all_actions:                
                item, extras = win32gui_struct.PackMENUITEMINFO(text=option_text,
                                                                hbmpItem=option_icon,
                                                                wID=option_id)
                win32gui.InsertMenuItem(menu, 0, 1, item)
            else:
                submenu = win32gui.CreatePopupMenu()
                self.create_menu(submenu, option_action)
                item, extras = win32gui_struct.PackMENUITEMINFO(text=option_text,
                                                                hbmpItem=option_icon,
                                                                hSubMenu=submenu)
                win32gui.InsertMenuItem(menu, 0, 1, item)

    def prep_menu_icon(self, icon):
        # First load the icon.
        ico_x = win32api.GetSystemMetrics(win32con.SM_CXSMICON)
        ico_y = win32api.GetSystemMetrics(win32con.SM_CYSMICON)
        hicon = win32gui.LoadImage(0, icon, win32con.IMAGE_ICON, ico_x, ico_y, win32con.LR_LOADFROMFILE)

        hdcBitmap = win32gui.CreateCompatibleDC(0)
        hdcScreen = win32gui.GetDC(0)
        hbm = win32gui.CreateCompatibleBitmap(hdcScreen, ico_x, ico_y)
        hbmOld = win32gui.SelectObject(hdcBitmap, hbm)
        # Fill the background.
        brush = win32gui.GetSysColorBrush(win32con.COLOR_MENU)
        win32gui.FillRect(hdcBitmap, (0, 0, 16, 16), brush)
        # unclear if brush needs to be feed.  Best clue I can find is:
        # "GetSysColorBrush returns a cached brush instead of allocating a new
        # one." - implies no DeleteObject
        # draw the icon
        win32gui.DrawIconEx(hdcBitmap, 0, 0, hicon, ico_x, ico_y, 0, 0, win32con.DI_NORMAL)
        win32gui.SelectObject(hdcBitmap, hbmOld)
        win32gui.DeleteDC(hdcBitmap)
        
        return hbm

    def command(self, hwnd, msg, wparam, lparam):
        id = win32gui.LOWORD(wparam)
        self.execute_menu_option(id)
        
    def execute_menu_option(self, id):
        menu_action = self.all_actions[id]      
        if menu_action == self.QUIT:
            win32gui.DestroyWindow(self.hwnd)
        else:
            menu_action(self)
            
def non_string_iterable(obj):
    try:
        iter(obj)
    except TypeError:
        return False
    else:
        return not isinstance(obj, basestring)


In [13]:
def g(num, source, msg):
    print num, source, msg.decode('gbk')

In [3]:
from sine.threads import ReStartableThread
_data = {'instance':None, 'exist':False}

def createIcon():
    def mainLoop(stop_event):
        _data['instance'] = SysTrayIcon('', 'hover', (), window_class_name='a')
        _data['exist'] = True
        win32gui.PumpMessages()
    trayThread = ReStartableThread(target=mainLoop)
    trayThread.start()

In [7]:
createIcon()

In [10]:
hwnd = _data['instance'].hwnd

In [11]:
try:
    print win32gui.PostMessage(hwnd, win32con.WM_CLOSE, 0, 0)
    win32gui.DestroyWindow(hwnd)
except Exception, e:
    if e[0] == 1400:
        raise

error: (1400, 'PostMessage', '\xce\xde\xd0\xa7\xb5\xc4\xb4\xb0\xbf\xda\xbe\xe4\xb1\xfa\xa1\xa3')

In [17]:
g(-2147467259, 'Shell_NotifyIcon', ('\xce\xb4\xd6\xb8\xb6\xa8\xb5\xc4\xb4\xed\xce\xf3'))

-2147467259 Shell_NotifyIcon 未指定的错误


In [14]:
g(1410, 'RegisterClass', '\xc0\xe0\xd2\xd1\xb4\xe6\xd4\xda\xa1\xa3')

1410 RegisterClass 类已存在。


In [15]:
g(5, 'DestroyWindow', '\xbe\xdc\xbe\xf8\xb7\xc3\xce\xca\xa1\xa3')

5 DestroyWindow 拒绝访问。


In [18]:
g(1412, 'UnregisterClass', '\xc0\xe0\xc8\xd4\xd3\xd0\xb4\xf2\xbf\xaa\xb5\xc4\xb4\xb0\xbf\xda\xa1\xa3')

1412 UnregisterClass 类仍有打开的窗口。


In [ ]:
g('\xce\xde\xd0\xa7\xb5\xc4\xb4\xb0\xbf\xda\xbe\xe4\xb1\xfa\xa1\xa3')#1400

In [ ]:
g('\xc0\xe0\xc8\xd4\xd3\xd0\xb4\xf2\xbf\xaa\xb5\xc4\xb4\xb0\xbf\xda\xa1\xa3')#1412

In [ ]:
(1407, 'CreateWindow', g('\xd5\xd2\xb2\xbb\xb5\xbd\xb4\xb0\xbf\xda\xc0\xe0\xa1\xa3'))

In [ ]:
1411, 'UnregisterClass', g('\xc0\xe0\xb2\xbb\xb4\xe6\xd4\xda\xa1\xa3')

In [ ]:
g('\xc0\xe0\xc8\xd4\xd3\xd0\xb4\xf2\xbf\xaa\xb5\xc4\xb4\xb0\xbf\xda\xa1\xa3')

In [ ]:
g('\xc0\xe0\xb2\xbb\xb4\xe6\xd4\xda\xa1\xa3')

In [ ]:
g('\xce\xde\xd0\xa7\xb5\xc4\xb4\xb0\xbf\xda\xbe\xe4\xb1\xfa\xa1\xa3')

In [ ]:
print '\xc0\xe0\xc8\xd4\xd3\xd0\xb4\xf2\xbf\xaa\xb5\xc4\xb4\xb0\xbf\xda\xa1\xa3'.decode('gbk')

In [ ]:
print '\xce\xb4\xd6\xb8\xb6\xa8\xb5\xc4\xb4\xed\xce\xf3'.decode('gbk')

In [ ]:
'\xce'.decode('ascii')

In [12]:
import threading

In [ ]:
threading.Thread(target=)

In [14]:
e=Exception('3', u'我', '2')

In [15]:
e.args

('3', u'\u6211', '2')

In [ ]:

class ClientException(Exception):
    def __str__(self):
        

In [17]:
' '.join(())

''

In [2]:
u'鎸夊洖杞︾户缁€傘€傘€'.encode('utf8').decode('gbk')

UnicodeDecodeError: 'gbk' codec can't decode bytes in position 4-5: illegal multibyte sequence